# Segmentation

In [6]:
import os
from modalities.test_mask_segment import  segment_brain_images, mask_brain_images
from modalities.report_segmentation import traverse_and_generate_report

def run_segmentation_and_masking(input_folder: str, tpm_path: str):
    # searching for .nii files within the subdirectories of input_folder
    subject_ids = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.nii'):
                subject_ids.append(os.path.join(root, file))
    
    print(subject_ids)  # check .nii files

    for input_file in subject_ids:
        subject_id = os.path.basename(input_file).split('.')[0]
        subject_folder = os.path.dirname(input_file)
        segmentation_folder = os.path.join(subject_folder, 'segmentation')
        os.makedirs(segmentation_folder, exist_ok=True)
        
        for iteration in range(1, 4):
            segmentation_output = segment_brain_images(input_file, tpm_path, iteration, subject_id, subject_folder)
            if not segmentation_output:
                print(f"Segmentation failed for {input_file} in iteration {iteration}")
                break
            mask_output = mask_brain_images(os.path.join(segmentation_folder, f'{iteration}_segment'), subject_folder, subject_id, iteration)
            input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'masked_{subject_id}.nii')
            
            # renaming the masked output to the original file name for the next iter
            next_input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'{subject_id}.nii') 
            os.rename(input_file, next_input_file)
            input_file = next_input_file
    
    # Generate the report after all subjects have been processed
    traverse_and_generate_report(input_folder, input_folder)

# Example usage:
input_folder = 'data/analysis_conditions/'
tpm_path = 'data/TPM'
run_segmentation_and_masking(input_folder, tpm_path)

['data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/T2_176.nii', 'data/analysis_conditions/Treated_by_psilocin_analysis/T2_177/T2_177.nii', 'data/analysis_conditions/Treated_by_psilocin_analysis/T2_175/T2_175.nii', 'data/analysis_conditions/Treated_by_psilocin_analysis/T2_174/T2_174.nii', 'data/analysis_conditions/Treated_by_psilocin_analysis/T2_178/T2_178.nii', 'data/analysis_conditions/Treated_by_psilocin_analysis/T2_201/T2_201.nii', 'data/analysis_conditions/control_analysis/T2_227/T2_227.nii', 'data/analysis_conditions/control_analysis/T2_223/T2_223.nii', 'data/analysis_conditions/control_analysis/T2_226/T2_226.nii', 'data/analysis_conditions/control_analysis/T2_228/T2_228.nii', 'data/analysis_conditions/control_analysis/T2_225/T2_225.nii', 'data/analysis_conditions/control_analysis/T2_224/T2_224.nii']
240711-13:04:47,91 nipype.workflow INFO:
	 Workflow vbm_segment_1_T2_176 settings: ['check', 'execution', 'logging', 'monitoring']
240711-13:04:47,101 nipype.workflow INFO

stty: 'standard input': Inappropriate ioctl for device


240711-13:05:41,161 nipype.workflow INFO:
	 Workflow vbm_segment_2_T2_176 settings: ['check', 'execution', 'logging', 'monitoring']
240711-13:05:41,164 nipype.workflow INFO:
	 Running serially.
240711-13:05:41,165 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_2_T2_176.segment_T2_176_2" in "/home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/2_segment/vbm_segment_2_T2_176/segment_T2_176_2".
240711-13:05:41,168 nipype.workflow INFO:
	 [Node] Executing "segment_T2_176_2" <nipype.interfaces.spm.preprocess.NewSegment>
240711-13:06:16,208 nipype.workflow INFO:
	 [Node] Finished "segment_T2_176_2", elapsed time 35.039662s.
240711-13:06:16,212 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_2_T2_176.data_sink_T2_176_2" in "/home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/2_segment/vbm_segment_2_T2_176/data_sink_T2_176_2".
240711-13:06:16,220 nipype.workflow INFO:
	 [Node] Exec

stty: 'standard input': Inappropriate ioctl for device


240711-13:06:16,912 nipype.workflow INFO:
	 Workflow vbm_segment_3_T2_176 settings: ['check', 'execution', 'logging', 'monitoring']
240711-13:06:16,917 nipype.workflow INFO:
	 Running serially.
240711-13:06:16,918 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_3_T2_176.segment_T2_176_3" in "/home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/vbm_segment_3_T2_176/segment_T2_176_3".
240711-13:06:16,921 nipype.workflow INFO:
	 [Node] Executing "segment_T2_176_3" <nipype.interfaces.spm.preprocess.NewSegment>
240711-13:06:53,874 nipype.workflow INFO:
	 [Node] Finished "segment_T2_176_3", elapsed time 36.952624s.
240711-13:06:53,880 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_3_T2_176.data_sink_T2_176_3" in "/home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/vbm_segment_3_T2_176/data_sink_T2_176_3".
240711-13:06:53,884 nipype.workflow INFO:
	 [Node] Exec

...

240711-13:29:54,673 nipype.workflow INFO:
	 Workflow vbm_segment_2_T2_224 settings: ['check', 'execution', 'logging', 'monitoring']
240711-13:29:54,676 nipype.workflow INFO:
	 Running serially.
240711-13:29:54,677 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_2_T2_224.segment_T2_224_2" in "/home/sharapova/vbm_pipline/data/analysis_conditions/control_analysis/T2_224/segmentation/2_segment/vbm_segment_2_T2_224/segment_T2_224_2".
240711-13:29:54,680 nipype.workflow INFO:
	 [Node] Executing "segment_T2_224_2" <nipype.interfaces.spm.preprocess.NewSegment>
240711-13:30:42,282 nipype.workflow INFO:
	 [Node] Finished "segment_T2_224_2", elapsed time 47.598968s.
240711-13:30:42,286 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_2_T2_224.data_sink_T2_224_2" in "/home/sharapova/vbm_pipline/data/analysis_conditions/control_analysis/T2_224/segmentation/2_segment/vbm_segment_2_T2_224/data_sink_T2_224_2".
240711-13:30:42,291 nipype.workflow INFO:
	 [Node] Executing "data_sink_T2_224_

stty: 'standard input': Inappropriate ioctl for device


240711-13:30:42,941 nipype.workflow INFO:
	 Workflow vbm_segment_3_T2_224 settings: ['check', 'execution', 'logging', 'monitoring']
240711-13:30:42,945 nipype.workflow INFO:
	 Running serially.
240711-13:30:42,945 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_3_T2_224.segment_T2_224_3" in "/home/sharapova/vbm_pipline/data/analysis_conditions/control_analysis/T2_224/segmentation/3_segment/vbm_segment_3_T2_224/segment_T2_224_3".
240711-13:30:42,949 nipype.workflow INFO:
	 [Node] Executing "segment_T2_224_3" <nipype.interfaces.spm.preprocess.NewSegment>
240711-13:31:21,33 nipype.workflow INFO:
	 [Node] Finished "segment_T2_224_3", elapsed time 38.082377s.
240711-13:31:21,38 nipype.workflow INFO:
	 [Node] Setting-up "vbm_segment_3_T2_224.data_sink_T2_224_3" in "/home/sharapova/vbm_pipline/data/analysis_conditions/control_analysis/T2_224/segmentation/3_segment/vbm_segment_3_T2_224/data_sink_T2_224_3".
240711-13:31:21,44 nipype.workflow INFO:
	 [Node] Executing "data_sink_T2_224_3" 

stty: 'standard input': Inappropriate ioctl for device


General report saved at: data/analysis_conditions/general_report.html


# Dartel

In [7]:
%load_ext autoreload
%autoreload 2
import os
from modalities.dartel import create_dartel_structure, perform_dartel, copy_segment_files, find_subject_id

def main(base_path: str):
    condition_folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    dartel_dir = create_dartel_structure(base_path)
    copy_segment_files(base_path, condition_folders, dartel_dir)
    perform_dartel(dartel_dir)

main('/home/sharapova/vbm_pipline/data/analysis_conditions')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c2/c2T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/rc1/rc1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/

stty: 'standard input': Inappropriate ioctl for device


# Normalization

In [8]:
%load_ext autoreload
%autoreload 2
import os
from modalities.normalization import normalize_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')
output_files = normalize_brain_images(root_derivatives_dir_path)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c1_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_225.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_226.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_227.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_228.nii']
flow_field_files: ['/home/sharap

stty: 'standard input': Inappropriate ioctl for device


# Smoothing

In [9]:
%load_ext autoreload
%autoreload 2
import os
from modalities.smoothing import smooth_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')

# Define the full width at half maximum (FWHM) for smoothing
fwhm = [3, 3, 3] 


output_files = smooth_brain_images(root_derivatives_dir_path, fwhm)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
normalize_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_225.nii', '/home

240711-13:55:24,809 nipype.workflow INFO:
	 [Node] Finished "smoothing", elapsed time 14.870629s.
240711-13:55:24,812 nipype.workflow INFO:
	 [Node] Setting-up "vbm_smoothing.data_sink" in "/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/work/vbm_smoothing/data_sink".
240711-13:55:24,815 nipype.workflow INFO:
	 [Node] Executing "data_sink" <nipype.interfaces.io.DataSink>
240711-13:55:24,816 nipype.workflow INFO:
	 [Node] Finished "data_sink", elapsed time 0.000735s.
['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_227.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_225.nii', '/home/sharapova/vbm_pipline/data/analysis_condition

stty: 'standard input': Inappropriate ioctl for device


# Statistical modeling and estimation

In [10]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_modeling_t_test_model import create_two_sample_t_test_model

contrasts_name = 'example_contrast_new_logic'
output_files = create_two_sample_t_test_model('/home/sharapova/vbm_pipline/data/analysis_conditions', contrasts_name)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Detected conditions: ['Treated_by_psilocin_analysis', 'control_analysis']
Detected subjects for Treated_by_psilocin_analysis: ['T2_176', 'T2_177', 'T2_174', 'T2_178', 'T2_201']
Detected subjects for control_analysis: ['T2_227', 'T2_223', 'T2_226', 'T2_228', 'T2_225', 'T2_224']
240711-13:56:31,739 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.10, cwd=/home/sharapova/vbm_pipline)
240711-13:56:31,744 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_stats_modeling
240711-13:56:31,746 nipype.workflow DEBUG:
	 expanding workflow: vbm_stats_modeling
240711-13:56:31,746 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.vbm_stats_modeling
240711-13:56:31,747 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.data_sink
240711-13:56:31,747 nipype.workflow DEBUG:
	 finished expanding workflow: vbm_stats_modeling
240711-13:56:31,748 nipype.workflow INFO:
	

stty: 'standard input': Inappropriate ioctl for device


240711-13:56:43,763 nipype.workflow INFO:
	 [Job 0] Completed (vbm_stats_modeling.vbm_stats_modeling).
240711-13:56:43,776 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
240711-13:56:43,779 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.10/28.10, Free processors: 4/4.
240711-13:56:43,838 nipype.workflow DEBUG:
	 Allocating vbm_stats_modeling.data_sink ID=1 (0.20GB, 1 threads). Free: 27.90GB, 3 threads.
240711-13:56:43,838 nipype.workflow DEBUG:
	 [Node] Not cached "/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/work/vbm_stats_modeling/data_sink".
240711-13:56:43,839 nipype.workflow DEBUG:
	 Checking hash "vbm_stats_modeling.data_sink" locally: cached=False, updated=False.
240711-13:56:43,841 nipype.workflow DEBUG:
	 [MultiProc] Submitted task vbm_stats_modeling.data_sink (taskid=2).
240711-13:56:43,843 nipype.workflow INFO:
	 [Node] Setting-up "vbm_stats_modeling.data_sink" in "/home/sharapova/vbm_pipline/data/

stty: 'standard input': Inappropriate ioctl for device


240711-13:57:03,807 nipype.workflow INFO:
	 [Job 0] Completed (vbm_estimate_model.estimate_model).
240711-13:57:03,811 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
240711-13:57:03,814 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 28.10/28.10, Free processors: 4/4.
240711-13:57:03,815 nipype.workflow DEBUG:
	 No tasks are being run, and no jobs can be submitted to the queue. Potential deadlock
['/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/RPV.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/mask.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/ResMS.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0001.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/SPM.mat', '/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0002.nii']
